# Label side

In [2]:
import numpy as np
from jesse import helpers, research

_, trading_1m = research.get_candles(
    "Binance Perpetual Futures",
    "BTC-USDT",
    "1m",
    helpers.date_to_timestamp("2020-01-01"),
    helpers.date_to_timestamp("2025-04-10"),
    warmup_candles_num=0,
    caching=False,
    is_for_jesse=False,
)

trading_1m.shape

(2773440, 6)

In [3]:
np.save("data/btc_1m.npy", trading_1m)

In [5]:
import numpy as np

trading_1m = np.load("data/btc_1m.npy")
trading_1m.shape

(2773440, 6)

In [6]:
from custom_indicators.toolbox.bar.dollar_bar import estimate_dollar_bar_threshold

for i in [80, 240, 720]:
    threshold = estimate_dollar_bar_threshold(trading_1m, i)
    print(f"{i}m: {threshold}")

80m: 698822832.1526461
240m: 2141074207.0847027
720m: 6423222619.254108


In [7]:
from jesse import helpers

from custom_indicators.toolbox.bar.dollar_bar import build_dollar_bar
from strategies.BinanceMLV2.config import DOLLAR_BAR_THRESHOLD_MID

print(DOLLAR_BAR_THRESHOLD_MID)

tp = helpers.date_to_timestamp("2025-04-05")

label_bar = build_dollar_bar(trading_1m, DOLLAR_BAR_THRESHOLD_MID)

print(label_bar.shape)

2141074207.0847027
(11540, 6)


In [8]:
from custom_indicators.toolbox.labeler.zigzag_labeler import zigzag

res = zigzag(label_bar[:, 2], 0.025)
np.unique(res, return_counts=True)

(array([-1,  1]), array([5471, 6069]))

In [9]:
label = np.column_stack((label_bar, res))
label

array([[ 1.57791846e+12,  7.19586000e+03,  7.35300000e+03, ...,
         6.86344000e+03,  2.98961076e+05,  1.00000000e+00],
       [ 1.57807746e+12,  7.35300000e+03,  7.50408000e+03, ...,
         7.25584000e+03,  2.89646946e+05,  1.00000000e+00],
       [ 1.57828878e+12,  7.50408000e+03,  7.84237000e+03, ...,
         7.48174000e+03,  2.76068141e+05,  1.00000000e+00],
       ...,
       [ 1.74422418e+12,  8.22447000e+04,  8.22914000e+04, ...,
         8.17383000e+04,  2.61669140e+04, -1.00000000e+00],
       [ 1.74422880e+12,  8.22914000e+04,  8.32880000e+04, ...,
         8.19801000e+04,  2.60240760e+04, -1.00000000e+00],
       [ 1.74423354e+12,  8.32881000e+04,  8.25880000e+04, ...,
         8.25000000e+04,  2.59203320e+04, -1.00000000e+00]])

In [10]:
np.save("data/label_side.npy", label[label[:, 0] <= tp])
print(label.shape)
print(label[label[:, 0] <= tp].shape)

(11540, 7)
(11470, 7)


In [11]:
from jesse import utils

df_dollar_bar = utils.numpy_candles_to_dataframe(
    label_bar[label_bar[:, 0] <= tp], name_date="time"
)
df_dollar_bar["zigzag"] = res[label_bar[:, 0] <= tp]
df_dollar_bar.head(1)

,time,open,close,high,low,volume,zigzag
2020-01-01 22:41:00,2020-01-01 22:41:00,7195.86,7353.0,7407.28,6863.44,298961.076,1


In [12]:
from lightweight_charts import Chart

chart = Chart(inner_width=1, inner_height=0.8)
chart2 = chart.create_subchart(position="bottom", width=1, height=0.2, sync=True)

chart.set(df_dollar_bar[["time", "open", "high", "low", "close", "volume"]])

line = chart2.create_line(name="zigzag", style="dotted")
line.set(df_dollar_bar[["time", "zigzag"]])

chart.show(block=True)

RuntimeError: asyncio.run() cannot be called from a running event loop

# Label meta

In [1]:
import lightgbm as lgb
import numpy as np
import pandas as pd

from strategies.BinanceMLV2.config import SIDE_ALL

label_side_1h = np.load("data/label_side.npy")
print(label_side_1h.shape)
print(np.unique(label_side_1h[:, 6], return_counts=True))

df_features = pd.read_parquet("data/features_240m.parquet")
print(df_features.shape)

side_model = lgb.Booster(model_file="strategies/BinanceMLV2/model/model_side.txt")
side_model_pred_prob = side_model.predict(df_features[SIDE_ALL])
df_features["side_model_res"] = side_model_pred_prob
side_model_pred = np.where(side_model_pred_prob > 0.5, 1, -1)
print(np.unique(side_model_pred, return_counts=True))

(11470, 7)
(array([-1.,  1.]), array([5415, 6055]))
(11470, 15051)
(array([-1,  1]), array([5613, 5857]))


In [2]:
meta_label = (side_model_pred.astype(int) == label_side_1h[:, 6].astype(int)).astype(
    int
)
np.unique(meta_label, return_counts=True)

(array([0, 1]), array([1944, 9526]))

In [3]:
label_meta_1h = label_side_1h.copy()
label_meta_1h[:, 6] = meta_label
np.save("data/label_meta.npy", label_meta_1h)